In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.torch_geo_models import GCN
from src.data.node_classifier.arxiv import load_dataset_pyg, data_to_sparse_symmetric_pyg

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset_pyg()

In [7]:
dataset.get_idx_split()

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [8]:
data = data_to_sparse_symmetric_pyg(dataset[0])

In [9]:
split_idx = dataset.get_idx_split()

In [10]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [11]:
features = data.x.cuda()
labels = data.y.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [12]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [13]:
data.adj_t = data.adj_t.to(device)

In [14]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [15]:
model = GCN(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout)\
    .to(device)

In [16]:
def train(model, graph, features, train_mask, optimizer, edge_weight=None):
    model.train()

    optimizer.zero_grad()
    out = model(features, graph, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [17]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=None):
    model.eval()

    out = model(features, graph, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [18]:
evaluator = Evaluator(name='ogbn-arxiv')

In [19]:
edge_weights = None

In [20]:
train_mask

tensor([     0,      1,      2,  ..., 169145, 169148, 169251], device='cuda:0')

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data.adj_t, features, train_mask, optimizer, edge_weight=edge_weights)
    result = test(model, data.adj_t, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=edge_weights)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.4290, Train: 62.81%, Valid: 63.52% Test: 61.81%
Epoch: 200, Loss: 1.1822, Train: 67.83%, Valid: 67.93% Test: 66.43%
Epoch: 300, Loss: 1.0985, Train: 69.73%, Valid: 69.34% Test: 68.12%
Epoch: 400, Loss: 1.0515, Train: 70.71%, Valid: 70.13% Test: 68.80%
Epoch: 500, Loss: 1.0195, Train: 71.42%, Valid: 70.60% Test: 69.15%
Epoch: 600, Loss: 0.9978, Train: 72.00%, Valid: 71.00% Test: 69.69%
Epoch: 700, Loss: 0.9749, Train: 72.50%, Valid: 71.25% Test: 69.98%
Epoch: 800, Loss: 0.9587, Train: 72.89%, Valid: 71.40% Test: 70.16%
Epoch: 900, Loss: 0.9393, Train: 73.21%, Valid: 71.45% Test: 70.10%
Epoch: 1000, Loss: 0.9282, Train: 73.51%, Valid: 71.63% Test: 70.17%
Epoch: 1100, Loss: 0.9144, Train: 73.84%, Valid: 71.72% Test: 70.46%
Epoch: 1200, Loss: 0.9044, Train: 74.09%, Valid: 71.77% Test: 70.58%
Epoch: 1300, Loss: 0.8927, Train: 74.38%, Valid: 71.91% Test: 70.58%
Epoch: 1400, Loss: 0.8820, Train: 74.59%, Valid: 72.02% Test: 70.70%
Epoch: 1500, Loss: 0.8728, Train: 74.90%, V